# Roll-your-own density functional theory

In [1]:
import numpy as np
from matplotlib import pyplot
%matplotlib inline

### The Helium atom

We first start with the H atom, for simplicity. No xc functional is needed, so we are just solving the electronic structure problem in a box with a potential defined by the point charge of the nucleus in the center of the box, at (0, 0, 0). Our objective is to write the Hamiltonian, which is (without nuclear-nuclear repulsion terms:

$$ \hat{h} = - \frac{\hbar^2}{2m} \nabla^2 - e^2 \sum_{i=1}^N \frac{Z}{ \left|  \vec{R} - \vec{r}_i \right| } + \frac{e^2}{2} \sum_{i=1}^N \sum_{j \ne i}^N \frac{1}{\left| \vec{r}_i - \vec{r}_j \right|} $$

We first set up the grid.

xs, ys, and zs are now flattened vectors, the elements of which correspond to the coordinates in 3D space of the original grid. Next, we define the potential.

In [2]:
g = 30  # Number of grid points in each dimension.
g3 = g**3.
p = np.linspace(-5., 5., g)
xs, ys, zs = np.meshgrid(p, p, p)
h = p[1] - p[0]  # Grid spacing.
xs = xs.flatten()
ys = ys.flatten()
zs = zs.flatten()

Define the constants. Initially, we start off with atomic units, so everything is unity. Later, we can add units.

In [3]:
hbar = 1.0
m = 1.0
e = 1.0

In [4]:
distances = np.sqrt( xs**2. + ys**2. + zs**2.)

Potential is now -2

In [5]:
Vext = -2.0/distances * e**2.

Next, we convert this to a sparse matrix

In [6]:
import scipy
from scipy import sparse
from scipy.sparse import linalg
Vext = sparse.spdiags(Vext,0,g3,g3)

(don't fully get this part yet, but we do understand that this gives charge neutrality within the Poisson equation, which makes zero density at the boundary condition less problematic) Here is the compensation charge:
$$ ncomp = \exp \left[ - \frac{R^2}{2} \right] $$
The corresponding potential is:
$$ ncomppot = -\frac{2}{R} \mathrm{erf} \left[ \frac{R}{\sqrt{2}} \right] $$

In [7]:
from scipy import special
ncomp = np.exp(-distances**2. / 2.0)
ncomp = -2.0 * ncomp / sum( ncomp ) / h**3.0
ncomppot = -2.0 / distances * scipy.special.erf( distances / np.sqrt(2.0))

Now we create the Laplacian. Start with the identity matrix.

In [8]:
I = sparse.eye(g)

The first part of the Laplacian, $[1, -2, 1]$. comes from the finite difference approximation for the second derivative:
$$ \frac{\partial^2 f \left( x \right) }{\partial x^2} \approx \frac{f_{i+1} - 2f_i + f_{i-1}}{\left( \Delta x \right)^2} $$
The first $L$ is our one-dimensional Laplacian.

In [9]:
_ = np.ones(g)
L = sparse.spdiags( [_, -2.0*_, _],[-1,0,1], g, g) / h**2.

(here we're using Andy's throwaway underscore instead of 'e', because using 'e' is silly.). The next bit of magic is the Kronecker tensor product to form the full 3D Laplacian.

In [15]:
L3 = sparse.kron( sparse.kron(L,I),I) + sparse.kron( sparse.kron(I,L),I) + sparse.kron( sparse.kron(I,I),L)
T = -0.5 * hbar**2. / m * L3 
L3.shape
ncomp.shape
n.shape

NameError: name 'n' is not defined

Form the Hamiltonian

In [50]:
def get_Vtot(Vguess):
    Hamiltonian = T + Vguess
    eigenvalues, eigenvectors = sparse.linalg.eigs(Hamiltonian,k=3, which='SR')
    #print eigenvectors[:,0].shape
    Psi = eigenvectors[:,0]/sum(eigenvectors[:,0])
    Psi /= h**(3./2.)
    #print Psi.shape
    n = 2.*Psi**2. #2 is for 2 electrons per orbital
    #print n.shape
    b=-4.0*np.pi * (n + ncomp)
    #print b.shape
    #print L3.shape
    Vx = - (3.0 / np.pi)**(1./3.) * n**(1./3.)
    Vh = sparse.linalg.cgs( A=L3, b=b, tol=1e-7,maxiter=400 ) 
    Vh = Vh[0] - ncomppot
    Vx = sparse.spdiags(Vx,0,g3,g3)
    Vh = sparse.spdiags(Vh,0,g3,g3)
    #print Vx.shape
    #print Vh.shape
    #print Vext.shape
    Vtot = Vx + Vh + Vext   
    return Vtot

In [51]:
Vtot = get_Vtot(Vext)
aaa

(27000, 27000)
(27000, 27000)
(27000, 27000)


NameError: name 'aaa' is not defined

In [ ]:

eigenvalues, eigenvectors = sparse.linalg.eigs(Hamiltonian,k=3, which='SR')

In [ ]:
print "energy of hydrogen atom in eV: %.3F"%(eigenvalues[0] * 27.21)
print eigenvectors

In [ ]:
sum(eigenvectors[1]**2.)

In [ ]:
eigenvectors[0]